### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
## Local Government Area Summary

# Total schools
totalSchools=len(school_data["School ID"].unique())
# Total students
totalStudents=len(student_data["Student ID"].drop_duplicates().dropna())
# Total budget
totalBudget=school_data["budget"].sum()

# Average maths score
averageMaths=student_data["maths_score"].mean()
# Average reading score
averageReading=student_data["reading_score"].mean()

# % passing maths (the percentage of students who passed maths)
passMaths_df=student_data["maths_score"] >= 50
passMathsPercent=passMaths_df.sum() / totalStudents * 100

# % passing reading (the percentage of students who passed reading)
passReading_df=student_data["reading_score"] >= 50
passReadingPercent=passReading_df.sum() / totalStudents * 100

# % overall passing (the percentage of students who passed maths AND reading)
passOverall_df=((student_data["maths_score"] >= 50 ) & (student_data["reading_score"] >= 50 ))
passOverallPercent=passOverall_df.sum() / totalStudents * 100

In [3]:
# Create a dataframe to hold the above results
summary=[{"Total Schools":totalSchools,"Total Students":totalStudents,"Total Budget":totalBudget,"Average Maths Score":averageMaths,\
    "Average Reading score":averageReading,"% Passing Maths":passMathsPercent,"% Passing Reading ":passReadingPercent,\
    "% Overall Passing":passOverallPercent}]
summary=pd.DataFrame(summary)

# Give the displayed data cleaner formatting
summary['Total Students']=summary['Total Students'].astype(int).map('{0:,.0f}'.format)
summary['Total Budget']=summary['Total Budget'].astype(int).map('${0:,.0f}'.format)

summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
## School Summary

# Make a school group group by school name
school_group=school_data_complete.groupby(["school_name"],as_index=False)

# Calc simple columns that can use the statistics function, and rename
school_summary=school_group.agg({'type':'unique','Student ID':'count','budget':'unique','maths_score':'mean','reading_score':'mean'})\
    .rename(columns={"school_name":"School Name","Student ID":"Total Students","type":"School Type","budget":"Total School Budget",\
        "maths_score":"Average Maths Score","reading_score":"Average Reading Score"})

In [5]:
# Remove the special character
school_summary['School Type']=school_summary['School Type'].astype(str).str.replace(r'\[|\]|\'', '')
school_summary['Total School Budget']=school_summary['Total School Budget'].astype(str).str.replace(r'\[|\]|\.', '')

# Format the budget with '$'
school_summary['Total School Budget']=school_summary['Total School Budget'].astype(int).map('${0:,.0f}'.format)

In [6]:
# Calc per student budget use apply()
def per_student_b(x):
    df = x['budget'].unique() / x['Student ID'].count()
    return df

per_student_budget=school_group.apply(per_student_b)

# Remove the special character
per_student_budget=per_student_budget.astype(str).str.replace(r'\[|\]|\.', '')
# Format the per student budget with '$'
per_student_budget=per_student_budget.astype(int).map('${0:,.0f}'.format)

In [7]:
# Calc percent of passing maths 
def percent_passing_m(x):
    df = (x['maths_score'] >= 50).sum() / x['Student ID'].count() * 100
    return df

percent_passing_maths=school_group.apply(percent_passing_m)

In [8]:
# Calc percent of passing reading
def percent_passing_r(x):
    df = (x['reading_score'] >= 50).sum() / x['Student ID'].count() * 100
    return df

percent_passing_reading=school_group.apply(percent_passing_r)

In [9]:
# Calc percent of overall passing
def percent_passing_o(x):
    df = ((x['maths_score'] >= 50) & (x['reading_score'] >= 50)).sum() / x['Student ID'].count() * 100
    return df

percent_passing_overall=school_group.apply(percent_passing_o)


In [10]:
# Make a New DataFrame add per student budget, passing maths, passing reading and overall passing
school_summary_group=school_summary.copy()
school_summary_group.insert(4,'Per Student Budget',per_student_budget)
school_summary_group['% Passing Maths']=percent_passing_maths
school_summary_group['% Passing Reading']=percent_passing_reading
school_summary_group['% Overall Passing']=percent_passing_overall

school_summary_group.style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928",$628,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356",$582,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411",$639,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916",$644,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500",$625,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020",$652,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087",$581,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635",$655,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650",$650,68.843100,69.039277,82.062592,81.978576,67.191766
Pena High School,Independent,962,"$585,858",$609,72.088358,71.613306,91.683992,86.590437,79.209979


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [11]:
## Top Performing Schools

# Sort by overall passing
top5=school_summary_group.sort_values("% Overall Passing",ascending=False).head(5)

#top5.reset_index(inplace=True, drop=True)
top5.style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500",$625,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356",$582,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928",$628,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400",$583,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363",$637,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [12]:
## Bottom Performing Schools

# Sort by overall passing with ascending=True
bottom5=school_summary_group.sort_values("% Overall Passing",ascending=True).head(5)

#bottom5.reset_index(inplace=True, drop=True)
bottom5.style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020",$652,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635",$655,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650",$650,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574",$578,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916",$644,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [13]:
## Maths Scores by Year 

# Make a maths group group by school name, year
yeargroup=school_data_complete.groupby(["school_name","year"],as_index=False)
# Average maths score
yearmaths=yeargroup.agg({'maths_score':'mean'})
# Get school name series
school_name=yearmaths['school_name'].unique()

# Get Year9, Year10, Year11, Year12's average maths score series
year9maths=yearmaths['maths_score'][yearmaths['year']==9]
year10maths=yearmaths['maths_score'][yearmaths['year']==10]
year11maths=yearmaths['maths_score'][yearmaths['year']==11]
year12maths=yearmaths['maths_score'][yearmaths['year']==12]

# Make a new DataFrame for output
data={'':school_name,'Year 9':year9maths.values,'Year 10':year10maths.values,'Year 11':year11maths.values,'Year 12':year12maths.values}
mathsummary=pd.DataFrame(data)

mathsummary.style.hide_index()

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Score by Year

* Perform the same operations as above for reading scores

In [14]:
## Maths Scores by Year 

# Make a reading group group by school name, year
yeargroup=school_data_complete.groupby(["school_name","year"],as_index=False)
# Average reading score
yearreading=yeargroup.agg({'reading_score':'mean'})
# Get school name series
school_name=yearreading['school_name'].unique()

# Get Year9, Year10, Year11, Year12's average reading score series
year9reading=yearreading['reading_score'][yearreading['year']==9]
year10reading=yearreading['reading_score'][yearreading['year']==10]
year11reading=yearreading['reading_score'][yearreading['year']==11]
year12reading=yearreading['reading_score'][yearreading['year']==12]

# Make a new DataFrame for output
data={'':school_name,'Year 9':year9reading.values,'Year 10':year10reading.values,'Year 11':year11reading.values,'Year 12':year12reading.values}
readingummary=pd.DataFrame(data)

readingummary.style.hide_index()

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [15]:
## Scores by School Spending

# Bin labels list
bin_labels=['<$585','$585-$630','$630-$645','$645-$680']
# Build a new DataFrame
school_spending = pd.DataFrame(school_summary_group, columns=['Per Student Budget','Average Maths Score','Average Reading Score',\
    '% Passing Maths','% Passing Reading','% Overall Passing'])
# remove special character
school_spending['Per Student Budget']=school_spending['Per Student Budget'].str.replace(r'$','')

# cut to 4 bins by per student budget
school_spending['Spending Range (Per Student)'] = pd.cut(school_spending['Per Student Budget'].astype(int),[500,585,630,645,680],labels=bin_labels)

# Group by spending range and average all
spending_group=school_spending.groupby(['Spending Range (Per Student)'])
spending=spending_group.mean().style.format('{0:,.2f}')
spending

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Range (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-$630,72.07,71.03,91.52,87.29,79.88
$630-$645,69.85,69.84,84.69,83.76,71.00
$645-$680,68.88,69.05,81.57,81.77,66.76


### Decription:
Based on the "Scores by School Spending", We can observe the costly school group having a poor average score and passing rate. The Best spending range __between $585 to $630__, with the highest score and highest passing rate. The second best school group, their budget is less than $585.

## Scores by School Size

* Perform the same operations as above, based on school size.

In [16]:
## Scores by School Spending

# Bin labels list
bin_labels=['Small (<1000)','Medium (1000-2000)','Large (2000-5000)	']
# Build a new DataFrame
school_size = pd.DataFrame(school_summary_group, columns=['Total Students','Average Maths Score','Average Reading Score',\
    '% Passing Maths','% Passing Reading','% Overall Passing'])

# Cut to 3 bins by per student budget
school_size['School Size'] = pd.cut(school_size['Total Students'].astype(int),[100,1000,2000,5000],labels=bin_labels)

# Group by school size and average all
size_group=school_size.groupby(['School Size'])
school_size=size_group[['Average Maths Score','Average Reading Score','% Passing Maths','% Passing Reading',\
    '% Overall Passing']].mean().style.format('{0:,.2f}')
school_size

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


### Decription:
We observe a trend that __small school__ group got the __best score__ and passing rate according "Scores by School Size", whatever maths score, reading score and all passing rate. The large school that more than 2000 students, got the worst score and passing rate.


## Scores by School Type

* Perform the same operations as above, based on school type

In [17]:
## Scores by School Spending

# Build a new DataFrame
school_type = pd.DataFrame(school_summary_group, columns=['School Type','Average Maths Score','Average Reading Score',\
    '% Passing Maths','% Passing Reading','% Overall Passing'])

# Group by school type and average all
school_type_group=school_type.groupby(['School Type'])
school_type=school_type_group.mean().style.format('{0:,.2f}')
school_type

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97


### Desciption:
The independent school have a better result than govenment school.